In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
    
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

# load up other dependencies
import re
import pandas as pd

import glob


In [ ]:
raw_data_files = glob.glob('*.gz')
raw_data_files

In [ ]:
base_df = spark.read.text(raw_data_files)
base_df.printSchema()

In [ ]:
type(base_df)

In [ ]:
base_df_rdd = base_df.rdd
type(base_df_rdd)

In [ ]:
base_df.show(10, truncate=False)

In [ ]:
base_df_rdd.take(10)

In [ ]:
print((base_df.count(), len(base_df.columns)))

In [ ]:
#Encontrando nome dos hots
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
hosts = [re.search(host_pattern, item).group(1)
           if re.search(host_pattern, item)
           else 'Nope'
           for item in sample_logs]

In [ ]:
base_df_rdd.show(10)

In [ ]:
##Extraindo as datas
ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'

timestamps = [re.search(ts_pattern, item).group(1) for item in sample_logs]

timestamps

In [ ]:
##Extraindo REQUESTS
method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'

method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()

               if re.search(method_uri_protocol_pattern, item)

               else 'Nope'

              for item in sample_logs]

method_uri_protocol

In [ ]:
# Extraindo o codigo do http
status_pattern = r'\s(\d{3})\s'

status = [re.search(status_pattern, item).group(1) for item in sample_logs]

print(status)

In [ ]:
##Unindo tudo
from pyspark.sql.functions import regexp_extract



logs_df = base_df.select(regexp_extract('value', host_pattern, 1).alias('host'),

                         regexp_extract('value', ts_pattern, 1).alias('timestamp'),

                         regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),

                         regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),

                         regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),

                         regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),

                         regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))

logs_df.show(10, truncate=True)

print((logs_df.count(), len(logs_df.columns)))

In [ ]:
#1 Contando os hosts unicos
unique_host_count = (logs_df.select('host').distinct().count())

In [ ]:
#2 - Reposta
(base_df .filter(base_df['status'] == '404').count())

In [ ]:
#3 - Resposta
hosts_404_count_df = (not_found_df.groupBy("host").count().sort("count", ascending=False).limit(5))

hosts_404_count_df.show(truncate=False)

In [ ]:
errors_by_date_sorted_df = (not_found_df.groupBy(F.dayofmonth('time').alias('day')).count().sort("day"))
errors_by_date_sorted_pd_df = errors_by_date_sorted_df.toPandas()
errors_by_date_sorted_pd_df